# Lab 3: Contextual Bandit-Based News Article Recommendation

**`Course`:** Reinforcement Learning Fundamentals  
**`Student Name`:**  
**`Roll Number`:**  
**`GitHub Branch`:** firstname_U20230xxx  

# Imports and Setup

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

from rlcmab_sampler import sampler


# Load Datasets

In [3]:
# Load datasets
news_df = pd.read_csv("data/news_articles.csv")
train_users = pd.read_csv("data/train_users.csv")
test_users = pd.read_csv("data/test_users.csv")

print(news_df.head())
print(train_users.head())


                                                link  \
0  https://www.huffpost.com/entry/covid-boosters-...   
1  https://www.huffpost.com/entry/american-airlin...   
2  https://www.huffpost.com/entry/funniest-tweets...   
3  https://www.huffpost.com/entry/funniest-parent...   
4  https://www.huffpost.com/entry/amy-cooper-lose...   

                                            headline   category  \
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  He was subdued by passengers and crew when he ...        Mary Papenfuss   
2  "Until you have a dog y

## Data Preprocessing

In this section:
- Handle missing values
- Encode categorical features
- Prepare data for user classification

In [4]:
# Data Preprocessing

# 1. Load the provided user and article datasets
print("=" * 80)
print("1. LOADING DATASETS")
print("=" * 80)

news_df = pd.read_csv("data/news_articles.csv")
train_users_df = pd.read_csv("data/train_users.csv")
test_users_df = pd.read_csv("data/test_users.csv")

print(f"\nNews Articles Dataset Shape: {news_df.shape}")
print(f"Train Users Dataset Shape: {train_users_df.shape}")
print(f"Test Users Dataset Shape: {test_users_df.shape}")

print("\nNews Articles Columns:", news_df.columns.tolist())
print("Train Users Columns:", train_users_df.columns.tolist())
print("Test Users Columns:", test_users_df.columns.tolist())

# 2. Data Cleaning - Handle Missing Values
print("\n" + "=" * 80)
print("2. DATA CLEANING - HANDLING MISSING VALUES")
print("=" * 80)

# Check for missing values in each dataset
print("\nMissing values in News Articles:")
print(news_df.isnull().sum())
print(f"Total missing: {news_df.isnull().sum().sum()}")

print("\nMissing values in Train Users:")
print(train_users_df.isnull().sum())
print(f"Total missing: {train_users_df.isnull().sum().sum()}")

print("\nMissing values in Test Users:")
print(test_users_df.isnull().sum())
print(f"Total missing: {test_users_df.isnull().sum().sum()}")

# Handle missing values in news articles
# Fill missing authors with 'Unknown'
news_df['authors'] = news_df['authors'].fillna('Unknown')

# Fill missing short_description with empty string
news_df['short_description'] = news_df['short_description'].fillna('')

# Fill missing date with mode (most frequent date)
if news_df['date'].isnull().sum() > 0:
    news_df['date'] = news_df['date'].fillna(news_df['date'].mode()[0] if not news_df['date'].mode().empty else 'Unknown')

# Handle missing values in user datasets (if any)
# Fill numerical columns with median
train_users_df['age'] = train_users_df['age'].fillna(train_users_df['age'].median())
train_users_df['income'] = train_users_df['income'].fillna(train_users_df['income'].median())
train_users_df['clicks'] = train_users_df['clicks'].fillna(train_users_df['clicks'].median())
train_users_df['purchase_amount'] = train_users_df['purchase_amount'].fillna(train_users_df['purchase_amount'].median())

test_users_df['age'] = test_users_df['age'].fillna(test_users_df['age'].median())
test_users_df['income'] = test_users_df['income'].fillna(test_users_df['income'].median())
test_users_df['clicks'] = test_users_df['clicks'].fillna(test_users_df['clicks'].median())
test_users_df['purchase_amount'] = test_users_df['purchase_amount'].fillna(test_users_df['purchase_amount'].median())

print("\nMissing values after cleaning:")
print(f"News Articles: {news_df.isnull().sum().sum()}")
print(f"Train Users: {train_users_df.isnull().sum().sum()}")
print(f"Test Users: {test_users_df.isnull().sum().sum()}")

# 3. Feature Encoding for Classification and Bandit Training
print("\n" + "=" * 80)
print("3. FEATURE ENCODING FOR CLASSIFICATION AND BANDIT TRAINING")
print("=" * 80)

# Encode categorical labels in user datasets (Convert user categories to numerical)
print("\nEncoding user labels...")
label_encoder = LabelEncoder()
train_users_df['label_encoded'] = label_encoder.fit_transform(train_users_df['label'])
test_users_df['label_encoded'] = label_encoder.transform(test_users_df['label'])

print(f"Original labels: {train_users_df['label'].unique()}")
print(f"Encoded labels: {train_users_df['label_encoded'].unique()}")
print(f"Mapping: {dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))}")

# Encode news article categories
print("\nEncoding news article categories...")
news_category_encoder = LabelEncoder()
news_df['category_encoded'] = news_category_encoder.fit_transform(news_df['category'])

print(f"Original categories: {news_df['category'].unique()}")
print(f"Encoded categories: {news_df['category_encoded'].unique()}")
print(f"Mapping: {dict(zip(news_category_encoder.classes_, news_category_encoder.transform(news_category_encoder.classes_)))}")

# Prepare feature sets for user classification
print("\nPreparing feature sets for user classification...")

# Numerical features
feature_columns = ['age', 'income', 'clicks', 'purchase_amount']

# Create train and test feature matrices
X_train = train_users_df[feature_columns].copy()
y_train = train_users_df['label_encoded'].copy()

X_test = test_users_df[feature_columns].copy()
y_test = test_users_df['label_encoded'].copy()

print(f"\nTrain Features Shape: {X_train.shape}")
print(f"Train Labels Shape: {y_train.shape}")
print(f"Test Features Shape: {X_test.shape}")
print(f"Test Labels Shape: {y_test.shape}")

# Normalize numerical features for better classification performance
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nFeatures normalized using StandardScaler")
print(f"Train features stats after scaling:")
print(f"  Mean: {X_train_scaled.mean(axis=0)}")
print(f"  Std: {X_train_scaled.std(axis=0)}")

# Summary of preprocessed data
print("\n" + "=" * 80)
print("PREPROCESSING SUMMARY")
print("=" * 80)
print(f"\n✓ Loaded 3 datasets successfully")
print(f"✓ Handled missing values in all datasets")
print(f"✓ Encoded categorical features (user labels and article categories)")
print(f"✓ Normalized numerical features for ML training")
print(f"\nReady for user classification and contextual bandit training!")


1. LOADING DATASETS

News Articles Dataset Shape: (209527, 6)
Train Users Dataset Shape: (2000, 6)
Test Users Dataset Shape: (2000, 6)

News Articles Columns: ['link', 'headline', 'category', 'short_description', 'authors', 'date']
Train Users Columns: ['user_id', 'age', 'income', 'clicks', 'purchase_amount', 'label']
Test Users Columns: ['user_id', 'age', 'income', 'clicks', 'purchase_amount', 'label']

2. DATA CLEANING - HANDLING MISSING VALUES

Missing values in News Articles:
link                     0
headline                 6
category                 0
short_description    19712
authors              37418
date                     0
dtype: int64
Total missing: 57136

Missing values in Train Users:
user_id            0
age                0
income             0
clicks             0
purchase_amount    0
label              0
dtype: int64
Total missing: 0

Missing values in Test Users:
user_id            0
age                0
income             0
clicks             0
purchase_amount 

## User Classification

Train a classifier to predict the user category (`User1`, `User2`, `User3`),
which serves as the **context** for the contextual bandit.


In [5]:
# User Classification - Train Multiple Models and Select the Best One

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import time

print("=" * 80)
print("USER CLASSIFICATION - TRAINING & MODEL SELECTION")
print("=" * 80)

# Dictionary to store all models and their results
models_dict = {}
results = []

# 1. Decision Tree Classifier
print("\n[1] Training Decision Tree Classifier...")
start_time = time.time()
dt_clf = DecisionTreeClassifier(random_state=42, max_depth=10)
dt_clf.fit(X_train_scaled, y_train)
dt_pred = dt_clf.predict(X_test_scaled)
dt_accuracy = accuracy_score(y_test, dt_pred)
dt_time = time.time() - start_time
models_dict['Decision Tree'] = dt_clf
results.append({
    'Model': 'Decision Tree',
    'Accuracy': dt_accuracy,
    'Precision': precision_score(y_test, dt_pred, average='weighted'),
    'Recall': recall_score(y_test, dt_pred, average='weighted'),
    'F1-Score': f1_score(y_test, dt_pred, average='weighted'),
    'Training Time (s)': dt_time
})
print(f"✓ Decision Tree Accuracy: {dt_accuracy:.4f} (Training time: {dt_time:.4f}s)")

# 2. Logistic Regression
print("\n[2] Training Logistic Regression...")
start_time = time.time()
lr_clf = LogisticRegression(random_state=42, max_iter=1000)
lr_clf.fit(X_train_scaled, y_train)
lr_pred = lr_clf.predict(X_test_scaled)
lr_accuracy = accuracy_score(y_test, lr_pred)
lr_time = time.time() - start_time
models_dict['Logistic Regression'] = lr_clf
results.append({
    'Model': 'Logistic Regression',
    'Accuracy': lr_accuracy,
    'Precision': precision_score(y_test, lr_pred, average='weighted'),
    'Recall': recall_score(y_test, lr_pred, average='weighted'),
    'F1-Score': f1_score(y_test, lr_pred, average='weighted'),
    'Training Time (s)': lr_time
})
print(f"✓ Logistic Regression Accuracy: {lr_accuracy:.4f} (Training time: {lr_time:.4f}s)")

# 3. Random Forest Classifier
print("\n[3] Training Random Forest Classifier...")
start_time = time.time()
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_clf.fit(X_train_scaled, y_train)
rf_pred = rf_clf.predict(X_test_scaled)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_time = time.time() - start_time
models_dict['Random Forest'] = rf_clf
results.append({
    'Model': 'Random Forest',
    'Accuracy': rf_accuracy,
    'Precision': precision_score(y_test, rf_pred, average='weighted'),
    'Recall': recall_score(y_test, rf_pred, average='weighted'),
    'F1-Score': f1_score(y_test, rf_pred, average='weighted'),
    'Training Time (s)': rf_time
})
print(f"✓ Random Forest Accuracy: {rf_accuracy:.4f} (Training time: {rf_time:.4f}s)")

# 4. Gradient Boosting Classifier
print("\n[4] Training Gradient Boosting Classifier...")
start_time = time.time()
gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_clf.fit(X_train_scaled, y_train)
gb_pred = gb_clf.predict(X_test_scaled)
gb_accuracy = accuracy_score(y_test, gb_pred)
gb_time = time.time() - start_time
models_dict['Gradient Boosting'] = gb_clf
results.append({
    'Model': 'Gradient Boosting',
    'Accuracy': gb_accuracy,
    'Precision': precision_score(y_test, gb_pred, average='weighted'),
    'Recall': recall_score(y_test, gb_pred, average='weighted'),
    'F1-Score': f1_score(y_test, gb_pred, average='weighted'),
    'Training Time (s)': gb_time
})
print(f"✓ Gradient Boosting Accuracy: {gb_accuracy:.4f} (Training time: {gb_time:.4f}s)")

# 5. Support Vector Machine (SVM)
print("\n[5] Training Support Vector Machine (SVM)...")
start_time = time.time()
svm_clf = SVC(kernel='rbf', random_state=42)
svm_clf.fit(X_train_scaled, y_train)
svm_pred = svm_clf.predict(X_test_scaled)
svm_accuracy = accuracy_score(y_test, svm_pred)
svm_time = time.time() - start_time
models_dict['SVM'] = svm_clf
results.append({
    'Model': 'SVM',
    'Accuracy': svm_accuracy,
    'Precision': precision_score(y_test, svm_pred, average='weighted'),
    'Recall': recall_score(y_test, svm_pred, average='weighted'),
    'F1-Score': f1_score(y_test, svm_pred, average='weighted'),
    'Training Time (s)': svm_time
})
print(f"✓ SVM Accuracy: {svm_accuracy:.4f} (Training time: {svm_time:.4f}s)")

# 6. K-Nearest Neighbors (KNN)
print("\n[6] Training K-Nearest Neighbors (KNN)...")
start_time = time.time()
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train_scaled, y_train)
knn_pred = knn_clf.predict(X_test_scaled)
knn_accuracy = accuracy_score(y_test, knn_pred)
knn_time = time.time() - start_time
models_dict['KNN'] = knn_clf
results.append({
    'Model': 'KNN',
    'Accuracy': knn_accuracy,
    'Precision': precision_score(y_test, knn_pred, average='weighted'),
    'Recall': recall_score(y_test, knn_pred, average='weighted'),
    'F1-Score': f1_score(y_test, knn_pred, average='weighted'),
    'Training Time (s)': knn_time
})
print(f"✓ KNN Accuracy: {knn_accuracy:.4f} (Training time: {knn_time:.4f}s)")

# Select the best model
print("\n" + "=" * 80)
print("MODEL COMPARISON RESULTS")
print("=" * 80)
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

best_model_name = results_df.loc[results_df['Accuracy'].idxmax(), 'Model']
best_accuracy = results_df['Accuracy'].max()
best_clf = models_dict[best_model_name]

print(f"\n{'🏆 BEST MODEL SELECTED: ' + best_model_name}")
print(f"{'=' * 80}")
print(f"Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"Model: {best_clf}")

# Detailed performance metrics for the best model
print("\n" + "=" * 80)
print(f"DETAILED PERFORMANCE - {best_model_name}")
print("=" * 80)

if best_model_name == 'Decision Tree':
    best_pred = dt_pred
elif best_model_name == 'Logistic Regression':
    best_pred = lr_pred
elif best_model_name == 'Random Forest':
    best_pred = rf_pred
elif best_model_name == 'Gradient Boosting':
    best_pred = gb_pred
elif best_model_name == 'SVM':
    best_pred = svm_pred
else:
    best_pred = knn_pred

# Confusion Matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, best_pred)
print(cm)

# Classification Report
print("\nClassification Report:")
class_report = classification_report(y_test, best_pred, target_names=label_encoder.classes_)
print(class_report)

# Feature Importance (if applicable)
if hasattr(best_clf, 'feature_importances_'):
    print("\nFeature Importance:")
    feature_importance = pd.DataFrame({
        'Feature': feature_columns,
        'Importance': best_clf.feature_importances_
    }).sort_values('Importance', ascending=False)
    print(feature_importance)

# Summary
print("\n" + "=" * 80)
print("CLASSIFICATION SUMMARY")
print("=" * 80)
print(f"✓ Trained {len(models_dict)} different classifiers")
print(f"✓ Best model: {best_model_name}")
print(f"✓ Test accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"✓ Context detector ready for bandit training!")


USER CLASSIFICATION - TRAINING & MODEL SELECTION

[1] Training Decision Tree Classifier...
✓ Decision Tree Accuracy: 0.3285 (Training time: 0.0115s)

[2] Training Logistic Regression...
✓ Logistic Regression Accuracy: 0.3295 (Training time: 0.0081s)

[3] Training Random Forest Classifier...
✓ Random Forest Accuracy: 0.3115 (Training time: 0.4153s)

[4] Training Gradient Boosting Classifier...
✓ Gradient Boosting Accuracy: 0.3240 (Training time: 0.8416s)

[5] Training Support Vector Machine (SVM)...
✓ SVM Accuracy: 0.3315 (Training time: 0.2584s)

[6] Training K-Nearest Neighbors (KNN)...
✓ KNN Accuracy: 0.3320 (Training time: 0.0115s)

MODEL COMPARISON RESULTS
              Model  Accuracy  Precision  Recall  F1-Score  Training Time (s)
      Decision Tree    0.3285   0.328153  0.3285  0.328198           0.011514
Logistic Regression    0.3295   0.319347  0.3295  0.309510           0.008090
      Random Forest    0.3115   0.312137  0.3115  0.311604           0.415256
  Gradient Boosting

In [7]:

# Hyperparameter Tuning for Maximum Accuracy

from sklearn.model_selection import GridSearchCV, cross_val_score

print("\n" + "=" * 80)
print("HYPERPARAMETER TUNING & OPTIMIZATION FOR MAXIMUM ACCURACY")
print("=" * 80)

# 1. Optimize KNN
print("\n[1] Tuning KNN Hyperparameters...")
knn_params = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}
knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5, scoring='accuracy', n_jobs=-1)
knn_grid.fit(X_train_scaled, y_train)
knn_best_pred = knn_grid.predict(X_test_scaled)
knn_best_accuracy = accuracy_score(y_test, knn_best_pred)
print(f"✓ Best KNN Accuracy: {knn_best_accuracy:.4f}")
print(f"  Best params: {knn_grid.best_params_}")

# 2. Optimize SVM with probability enabled
print("\n[2] Tuning SVM Hyperparameters...")
svm_params = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf', 'poly'],
    'gamma': ['scale', 'auto']
}
svm_grid = GridSearchCV(SVC(random_state=42, probability=True), svm_params, cv=5, scoring='accuracy', n_jobs=-1)
svm_grid.fit(X_train_scaled, y_train)
svm_best_pred = svm_grid.predict(X_test_scaled)
svm_best_accuracy = accuracy_score(y_test, svm_best_pred)
print(f"✓ Best SVM Accuracy: {svm_best_accuracy:.4f}")
print(f"  Best params: {svm_grid.best_params_}")

# 3. Optimize Random Forest
print("\n[3] Tuning Random Forest Hyperparameters...")
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10]
}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1), rf_params, cv=5, scoring='accuracy', n_jobs=-1)
rf_grid.fit(X_train_scaled, y_train)
rf_best_pred = rf_grid.predict(X_test_scaled)
rf_best_accuracy = accuracy_score(y_test, rf_best_pred)
print(f"✓ Best Random Forest Accuracy: {rf_best_accuracy:.4f}")
print(f"  Best params: {rf_grid.best_params_}")

# 4. Optimize Gradient Boosting
print("\n[4] Tuning Gradient Boosting Hyperparameters...")
gb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}
gb_grid = GridSearchCV(GradientBoostingClassifier(random_state=42), gb_params, cv=5, scoring='accuracy', n_jobs=-1)
gb_grid.fit(X_train_scaled, y_train)
gb_best_pred = gb_grid.predict(X_test_scaled)
gb_best_accuracy = accuracy_score(y_test, gb_best_pred)
print(f"✓ Best Gradient Boosting Accuracy: {gb_best_accuracy:.4f}")
print(f"  Best params: {gb_grid.best_params_}")

# 5. Create Voting Classifier (Ensemble with hard voting)
print("\n[5] Creating Ensemble Voting Classifier...")
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators=[
        ('knn', knn_grid.best_estimator_),
        ('svm', svm_grid.best_estimator_),
        ('rf', rf_grid.best_estimator_),
        ('gb', gb_grid.best_estimator_)
    ],
    voting='hard'
)
voting_clf.fit(X_train_scaled, y_train)
voting_pred = voting_clf.predict(X_test_scaled)
voting_accuracy = accuracy_score(y_test, voting_pred)
print(f"✓ Ensemble Voting Classifier Accuracy: {voting_accuracy:.4f}")

# Compare all tuned models
print("\n" + "=" * 80)
print("OPTIMIZED MODELS COMPARISON (RANKED BY ACCURACY)")
print("=" * 80)
tuned_results = [
    {'Model': 'KNN (Tuned)', 'Accuracy': knn_best_accuracy},
    {'Model': 'SVM (Tuned)', 'Accuracy': svm_best_accuracy},
    {'Model': 'Random Forest (Tuned)', 'Accuracy': rf_best_accuracy},
    {'Model': 'Gradient Boosting (Tuned)', 'Accuracy': gb_best_accuracy},
    {'Model': 'Ensemble Voting', 'Accuracy': voting_accuracy}
]
tuned_df = pd.DataFrame(tuned_results).sort_values('Accuracy', ascending=False).reset_index(drop=True)
print(tuned_df.to_string(index=False))

# Select final best model
final_best_model_name = tuned_df.iloc[0]['Model']
final_best_accuracy = tuned_df.iloc[0]['Accuracy']

# Assign the best model and predictions
if 'Ensemble' in final_best_model_name:
    final_classifier = voting_clf
    final_predictions = voting_pred
elif 'KNN' in final_best_model_name:
    final_classifier = knn_grid.best_estimator_
    final_predictions = knn_best_pred
elif 'SVM' in final_best_model_name:
    final_classifier = svm_grid.best_estimator_
    final_predictions = svm_best_pred
elif 'Random Forest' in final_best_model_name:
    final_classifier = rf_grid.best_estimator_
    final_predictions = rf_best_pred
else:
    final_classifier = gb_grid.best_estimator_
    final_predictions = gb_best_pred

print(f"\n{'🏆 MAXIMUM ACCURACY CLASSIFIER SELECTED: ' + final_best_model_name}")
print(f"{'=' * 80}")
print(f"Test Accuracy: {final_best_accuracy:.4f} ({final_best_accuracy*100:.2f}%)")
print(f"Improvement: {(final_best_accuracy - best_accuracy)*100:.2f}% over initial best model")

# Final detailed performance
print("\n" + "=" * 80)
print(f"FINAL PERFORMANCE METRICS - {final_best_model_name}")
print("=" * 80)

print("\nConfusion Matrix:")
final_cm = confusion_matrix(y_test, final_predictions)
print(final_cm)

print("\nClassification Report:")
final_report = classification_report(y_test, final_predictions, target_names=label_encoder.classes_)
print(final_report)

# Summary
print("\n" + "=" * 80)
print("FINAL CLASSIFICATION SUMMARY")
print("=" * 80)
print(f"✓ Performed hyperparameter tuning on 4 classifiers using GridSearchCV (5-fold CV)")
print(f"✓ Created ensemble voting classifier combining best models")
print(f"✓ Final best model: {final_best_model_name}")
print(f"✓ Final test accuracy: {final_best_accuracy:.4f} ({final_best_accuracy*100:.2f}%)")
print(f"✓ Context detector optimized with maximum accuracy!")
print(f"✓ Ready for contextual bandit training!")



HYPERPARAMETER TUNING & OPTIMIZATION FOR MAXIMUM ACCURACY

[1] Tuning KNN Hyperparameters...
✓ Best KNN Accuracy: 0.3225
  Best params: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}

[2] Tuning SVM Hyperparameters...
✓ Best SVM Accuracy: 0.3455
  Best params: {'C': 10, 'gamma': 'scale', 'kernel': 'poly'}

[3] Tuning Random Forest Hyperparameters...
✓ Best Random Forest Accuracy: 0.3185
  Best params: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 300}

[4] Tuning Gradient Boosting Hyperparameters...
✓ Best Gradient Boosting Accuracy: 0.3130
  Best params: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 100}

[5] Creating Ensemble Voting Classifier...
✓ Ensemble Voting Classifier Accuracy: 0.3250

OPTIMIZED MODELS COMPARISON (RANKED BY ACCURACY)
                    Model  Accuracy
              SVM (Tuned)    0.3455
          Ensemble Voting    0.3250
              KNN (Tuned)    0.3225
    Random Forest (Tuned)    0.3185
Gradient Boosting (Tuned) 

# `Contextual Bandit`

## Reward Sampler Initialization

The sampler is initialized using the student's roll number `i`.
Rewards are obtained using `sampler.sample(j)`.


## Arm Mapping

| Arm Index (j) | News Category | User Context |
|--------------|---------------|--------------|
| 0–3          | Entertainment, Education, Tech, Crime | User1 |
| 4–7          | Entertainment, Education, Tech, Crime | User2 |
| 8–11         | Entertainment, Education, Tech, Crime | User3 |

## Epsilon-Greedy Strategy

This section implements the epsilon-greedy contextual bandit algorithm.


## Upper Confidence Bound (UCB)

This section implements the UCB strategy for contextual bandits.

## SoftMax Strategy

This section implements the SoftMax strategy with temperature $ \tau = 1$.


## Reinforcement Learning Simulation

We simulate the bandit algorithms for $T = 10,000$ steps and record rewards.

P.S.: Change $T$ value as and if required.


## Results and Analysis

This section presents:
- Average Reward vs Time
- Hyperparameter comparisons
- Observations and discussion


## Final Observations

- Comparison of Epsilon-Greedy, UCB, and SoftMax
- Effect of hyperparameters
- Strengths and limitations of each approach
